In [1]:
#etiket verme

import pandas as pd
from transformers import pipeline


classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["Positive", "Negative", "Neutral"]

csv_file_path = 'youtube.csv'


df = pd.read_csv(csv_file_path)


print("Veri Seti:")
print(df.head())  

print("\nEksik değerler:")
print(df.isnull().sum())


df = df.dropna(subset=['comment'])

def classify_comment(comment):
    if isinstance(comment, str):
        result = classifier(
            comment, 
            candidate_labels, 
            hypothesis_template="The sentiment of this text is {}."
        )
 
        max_score = max(result["scores"])
        label = result["labels"][0]

        if max_score < 0.45: 
            label = "Neutral"
        return label, max_score
    return None, None


df[['sentiment', 'score']] = df['comment'].apply(lambda x: classify_comment(x)).apply(pd.Series)


print("\nEtiketlenmiş Veri:")
print(df.head())  


output_file_path = 'etkitelenen_veri.csv'
df.to_csv(output_file_path, index=False)

print(f"Etiketlenmiş veri seti '{output_file_path}' olarak kaydedildi.")


print("\nEtiket Frekansları:")
print(df['sentiment'].value_counts())


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


Veri Seti:
                                             comment
0  mum eriyince biter dedim 5 mum 3 gaz lambası 3...
1  Youtubeda o kadar detaylı anlatan hoca varken ...
2  Tersten de izledim integrali de hallettim hoca...
3   anne şu video bitsin öyle gideceğim ekmek almaya
4   Kesinlikle imdb top 5'e girer. Mükemmel bir film

Eksik değerler:
comment    0
dtype: int64

Etiketlenmiş Veri:
                                             comment sentiment     score
0  mum eriyince biter dedim 5 mum 3 gaz lambası 3...  Negative  0.653513
1  Youtubeda o kadar detaylı anlatan hoca varken ...  Negative  0.600888
2  Tersten de izledim integrali de hallettim hoca...  Negative  0.547657
3   anne şu video bitsin öyle gideceğim ekmek almaya  Positive  0.456882
4   Kesinlikle imdb top 5'e girer. Mükemmel bir film  Positive  0.635313
Etiketlenmiş veri seti 'etkitelenen_veri.csv' olarak kaydedildi.

Etiket Frekansları:
sentiment
Negative    577
Positive    348
Neutral     346
Name: count, dtype: int64
